# Introduction

After looking at the FutureCrop data, it appears that there's a large variance across locations in the mean yield and also the variance.

Here we will try to model each location independently- we then will try to scale this up to train the many models in parallel on a GPU using pytorch.

The idea is to learn the minimal mapping from time-series data to a prediction of the yield.

**Notes**:
A simple model just learns to predict the mean across all training batches of a single location. This is true for tiny (1-unit), shallow (100-unit wide) and deep networks (2-4 layers).

We need a smoother loss than MSE and some regularisation to improve on this.

In [60]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data_dir = '/kaggle/input/the-future-crop-challenge/'
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-future-crop-challenge/pr_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/tasmax_maize_train.parquet
/kaggle/input/the-future-crop-challenge/sample_submission.csv
/kaggle/input/the-future-crop-challenge/soil_co2_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/tas_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/rsds_maize_train.parquet
/kaggle/input/the-future-crop-challenge/tasmin_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/tasmax_wheat_train.parquet
/kaggle/input/the-future-crop-challenge/rsds_maize_test.parquet
/kaggle/input/the-future-crop-challenge/soil_co2_maize_test.parquet
/kaggle/input/the-future-crop-challenge/train_solutions_maize.parquet
/kaggle/input/the-future-crop-challenge/pr_maize_test.parquet
/kaggle/input/the-future-crop-challenge/tas_wheat_test.parquet
/kaggle/input/the-future-crop-challenge/tasmax_maize_test.parquet
/kaggle/input/the-future-crop-challenge/pr_maize_train.parquet
/kaggle/input/the-fu

In [61]:
#climate data (timeseries)

#soil_co2 and yields

wheat_sco2 = pd.read_parquet('/kaggle/input/the-future-crop-challenge/soil_co2_wheat_train.parquet')
wheat_yield = pd.read_parquet('/kaggle/input/the-future-crop-challenge/train_solutions_wheat.parquet')
wheat_df = wheat_sco2.join(wheat_yield)

maize_sco2 = pd.read_parquet('/kaggle/input/the-future-crop-challenge/soil_co2_maize_train.parquet')
maize_yield = pd.read_parquet('/kaggle/input/the-future-crop-challenge/train_solutions_maize.parquet')
maize_df = maize_sco2.join(maize_yield)

mean_df = pd.DataFrame()
for crop_df in [wheat_df,maize_df]:
    temp_df = crop_df.groupby(['crop','lon','lat'], as_index = False).agg({'yield':['mean','std']})
    temp_df.columns = ['crop','lon','lat','yield_mean','yield_std']
    mean_df = pd.concat([mean_df,temp_df])

mean_df


,crop,lon,lat,yield_mean,yield_std
0,wheat,-123.25,44.75,4.965216,0.488492
1,wheat,-123.25,45.25,4.985947,0.501791
2,wheat,-123.25,45.75,4.822316,0.423168
3,wheat,-122.75,44.75,4.875486,0.537611
4,wheat,-122.75,45.25,5.379421,0.615185
...,...,...,...,...,...
9298,maize,132.75,46.75,5.738692,1.191923
9299,maize,132.75,47.25,8.622872,1.518404
9300,maize,133.25,45.25,2.470256,0.359327
9301,maize,133.25,47.25,6.932128,1.339325


In [62]:
idx_lon = 132.75
idx_lat = 47.25

static_data = maize_df.query(f'lon=={idx_lon} and lat=={idx_lat}')[['co2','nitrogen','yield']]

crop = 'maize'
mode = 'train'

tasmax = pd.read_parquet(os.path.join(data_dir, f"tasmax_{crop}_{mode}.parquet")).query(f'lon=={idx_lon} and lat=={idx_lat}')
tasmin = pd.read_parquet(os.path.join(data_dir, f"tasmin_{crop}_{mode}.parquet")).query(f'lon=={idx_lon} and lat=={idx_lat}')
pr = pd.read_parquet(os.path.join(data_dir, f"pr_{crop}_{mode}.parquet")).query(f'lon=={idx_lon} and lat=={idx_lat}')
rsds = pd.read_parquet(os.path.join(data_dir, f"rsds_{crop}_{mode}.parquet")).query(f'lon=={idx_lon} and lat=={idx_lat}')

climate_data = np.stack([
        tasmax.iloc[:, 5:].values,
        tasmin.iloc[:, 5:].values,
        pr.iloc[:, 5:].values,
        rsds.iloc[:, 5:].values
    ], axis=2)

static_expanded = np.repeat(static_data.values[:,np.newaxis,:],240,axis=1)

sequenced_data = np.concatenate([climate_data,static_expanded],axis = 2)

sequenced_data.shape

(39, 240, 7)

In [63]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# --- Data Splitting ---
inputs = sequenced_data[:, :, :-1]
targets = sequenced_data[:, -1, -1]

# --- 1. Configuration ---
n_batch = sequenced_data.shape[0] # Using full dataset as one batch
n_seq = sequenced_data.shape[1]
n_features = sequenced_data.shape[2] - 1
n_hidden = 4
n_layers = 1
learning_rate = 1e-1
n_epochs = 1000

# Regularization Hyperparameter (Adjust if needed)
HIDDEN_REG_ALPHA = 1e-4 

# Device configuration (Move all data/model to the device)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"--- Configuration Summary ---")
print(f"Data Shape (Batch, Seq, Feat): ({n_batch}, {n_seq}, {n_features})")
print(f"Hidden Size/Layers: {n_hidden}/{n_layers}")
print(f"Learning Rate/Epochs: {learning_rate}/{n_epochs}")
print(f"Using Device: {device}")
print(f"-----------------------------")


# --- 2. PyTorch Dataset & DataLoader ---
class SequenceDataset(Dataset):
    def __init__(self, X, Y):
        # Move inputs to device and set dtype
        self.X = torch.from_numpy(X).to(device=device, dtype=torch.float32)
        # Move targets to device, set dtype, and unsqueeze to [batch, seq, 1] for loss calculation
        self.Y = torch.from_numpy(Y).to(device=device, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Since the DataLoader batch_size is the full dataset size, 
        # this will effectively return the entire X and Y tensors on the first call.
        return self.X[idx], self.Y[idx]

# Create DataLoader
dataset = SequenceDataset(inputs, targets)
# DataLoader batch_size is n_batch, so we get one large batch: (1, n_batch, n_seq, n_features)
dataloader = DataLoader(dataset, batch_size=n_batch, shuffle=False) 

# --- 3. Model Definition and Xavier Initialization ---

class SimpleRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SimpleRNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Using GRU as in the previous example
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
        self._init_weights()

    def _init_weights(self):
        # Xavier/Glorot Initialization
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 1)
        print("Model weights initialized with Xavier/Glorot.")

    def forward(self, x):
        # out: (batch_size, n_seq, hidden_size) - Sequence of hidden states
        out, _ = self.rnn(x)  
        
        # final_out: (batch_size, n_seq, output_size) - Final prediction
        final_out = self.fc(out)
        
        # Return both prediction and hidden states for regularization
        return final_out, out 

# Instantiate the model
model = SimpleRNNModel(
    input_size=n_features, 
    hidden_size=n_hidden, 
    num_layers=n_layers, 
    output_size=1
).to(device) # Ensure model is also on the device

# --- 4. Loss and Optimizer ---

# SmoothL1Loss for robustness
criterion_primary = nn.MSELoss().to(device) 
# AdamW Optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# --- 5. Training Loop ---

print("-" * 30)
print(f"Starting Training for {n_epochs} epochs...")
model.train() 

for epoch in range(1, n_epochs + 1):
    epoch_loss = 0
    
    for inputs_batch, targets_batch in dataloader:
        # Data is already on the device due to the Dataset implementation
        # Forward pass:
        outputs, hidden_states = model(inputs_batch)
        
        # 1. Primary Loss (Smooth L1 Loss)
        primary_loss = criterion_primary(outputs[:,-1,-1], targets_batch)

        # 2. Hidden State Regularization Loss (L2 Norm Squared)
        hidden_reg_loss = torch.norm(hidden_states, p=2)**2 * HIDDEN_REG_ALPHA
        
        # 3. Total Loss
        loss = primary_loss + hidden_reg_loss
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    # Reporting every 1000 epochs to avoid excessive output
    if epoch % 100 == 0 or epoch == 1:
        avg_loss = epoch_loss / len(dataloader)
        print(f'Epoch [{epoch:05d}/{n_epochs}], Total Loss: {avg_loss:.6f} (Primary Loss: {primary_loss.item():.6f}, Reg Loss: {hidden_reg_loss.item():.6f})')

print("-" * 30)
print("Training Complete!")

--- Configuration Summary ---
Data Shape (Batch, Seq, Feat): (39, 240, 6)
Hidden Size/Layers: 4/1
Learning Rate/Epochs: 0.1/1000
Using Device: cpu
-----------------------------
Model weights initialized with Xavier/Glorot.
------------------------------
Starting Training for 1000 epochs...
Epoch [00001/1000], Total Loss: 60.358364 (Primary Loss: 60.358360, Reg Loss: 0.000002)
Epoch [00100/1000], Total Loss: 4.114655 (Primary Loss: 2.246572, Reg Loss: 1.868084)
Epoch [00200/1000], Total Loss: 4.115640 (Primary Loss: 2.247556, Reg Loss: 1.868084)
Epoch [00300/1000], Total Loss: 4.115234 (Primary Loss: 2.247150, Reg Loss: 1.868084)
Epoch [00400/1000], Total Loss: 4.115030 (Primary Loss: 2.246946, Reg Loss: 1.868084)
Epoch [00500/1000], Total Loss: 4.114907 (Primary Loss: 2.246824, Reg Loss: 1.868084)
Epoch [00600/1000], Total Loss: 4.114826 (Primary Loss: 2.246742, Reg Loss: 1.868084)
Epoch [00700/1000], Total Loss: 4.114769 (Primary Loss: 2.246685, Reg Loss: 1.868084)
Epoch [00800/1000],

## Learning a time-series -> yield function

Here we're very data-limited using only a single location. There are 240x4+1 inputs, but only 39 training examples. 

1. DNN with dropout (for generalisation). With a VAE-like dimension reduction layer-by-layer.

One idea would be to do a convolution over the 240x4 inputs and reduce them to an N dimensional signal, which is then combined with the C02 data to produce a yield estimate. This might not be crazy at all.



In [67]:
# DNN with dropout
# simply map continous data onto the yield with an N-deep network trained with dropout.

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# --- 2. PyTorch Dataset & DataLoader (Updated for 1D target) ---
class FlattenedDataset(Dataset):
    def __init__(self, climate_data, soil_co2, crop_yield):
        """Takes numpy arrays as inputs.
        climate_data shaped as (n_batch, n_seq, n_features)
        soil_co2 and crop_yield both 1d arrays"""
        N, S, F = climate_data.shape
        # X: Flattened to [N, S * F]
        self.X = torch.from_numpy(climate_data).reshape(N, S * F)
        self.X = torch.concat([self.X,torch.from_numpy(soil_co2).unsqueeze(-1)],axis=1).to(device, dtype=torch.float32)
        # Y: Unsqueeze to [N, 1] for consistent loss calculation
        self.Y = torch.from_numpy(crop_yield).unsqueeze(-1).to(device=device, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# --- 3. Flexible Model Definition and Xavier Initialization (DNN) ---

class FlexibleFlattenedDNN(nn.Module):
    def __init__(self, input_size, output_size, num_layers, reduction_ratio, dropout_rate=0.0, leaky_relu_slope =0.01):
        super(FlexibleFlattenedDNN, self).__init__()
        self.leaky_slope = leaky_relu_slope
        layers = []
        current_size = input_size
        
        # Dynamically build the hidden layers
        for i in range(num_layers):
            # Calculate the size of the next layer
            next_size = max(4, int(current_size * reduction_ratio)) # Min size of 4 for stability
            
            # Add Linear Layer
            layers.append(nn.Linear(current_size, next_size))
            # Add Activation
            layers.append(nn.LeakyReLU(self.leaky_slope))
            
            # Add Dropout (only for intermediate layers)
            if dropout_rate > 0 and i < num_layers - 1:
                 layers.append(nn.Dropout(dropout_rate)) 
                
            current_size = next_size
        
        # Add the final output layer (no activation or dropout after this)
        layers.append(nn.Linear(current_size, output_size))
        
        self.fc_stack = nn.Sequential(*layers)
        
        print(f"DNN Architecture built: {input_size} -> {[l.out_features for l in layers if isinstance(l, nn.Linear)]}")
        
        self._init_weights()

    def _init_weights(self):
        # Xavier/Glorot Initialization for all Linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                #nn.init.xavier_uniform_(m.weight)
                nn.init.kaiming_normal_(m.weight, a= self.leaky_slope)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        print("Model weights initialized with Xavier/Glorot.")

    def forward(self, x):
        return self.fc_stack(x) 

# --- 1. Configuration ---
n_batch = sequenced_data.shape[0] 
n_seq = sequenced_data.shape[1]   
n_features = sequenced_data.shape[2] - 1

# --- MODEL FLEXIBILITY PARAMETERS ---
NUM_HIDDEN_LAYERS = 3     # The number of layers between input and output (e.g., 3 for 4 layers total)
REDUCTION_RATIO = 1/6      # The ratio by which each layer size decreases (e.g., 0.5 means half the size)
DROPOUT_RATE = 2/5        # Dropout rate for intermediate layers (0.0 for no dropout)

# --- CALCULATED DIMENSIONS ---
FLATTENED_INPUT_SIZE = n_seq * 4+1 
TARGET_OUTPUT_SIZE = 1 # Corrected to 1D output

# Hyperparameter
init_LR = 3e-9
max_LR = 9e-3
weight_decay = 1e-5 #suggested to be smaller for 'super-convergence' in OneCycleLR paper.
n_epochs = 5000


# Create DataLoader
dataset = FlattenedDataset(climate_data, static_data.co2.values,static_data['yield'].values)
n_train = int(dataset.X.shape[0]*0.8)
n_test = dataset.X.shape[0]-n_train

train_dataset = torch.utils.data.Subset(dataset, range(n_train))
val_dataset = torch.utils.data.Subset(dataset,range(n_train,n_train+n_test))

train_loader = DataLoader(train_dataset, batch_size=n_batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = n_batch, shuffle = False)

# Instantiate the flexible model
model = FlexibleFlattenedDNN(
    input_size=FLATTENED_INPUT_SIZE,
    output_size=TARGET_OUTPUT_SIZE,
    num_layers=NUM_HIDDEN_LAYERS,
    reduction_ratio=REDUCTION_RATIO,
    dropout_rate=DROPOUT_RATE
).to(device, dtype=torch.float32)

n_params = sum([p.numel() for p in model.parameters()])
print(f"Model instantiatied with {n_params} parameters.")
# --- 4. Loss and Optimizer ---

# Switched back to MSELoss as requested in the original code block
criterion_primary = nn.MSELoss().to(device) 
optimizer = optim.AdamW(model.parameters(), lr=init_LR, weight_decay = weight_decay)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_LR, 
                                          steps_per_epoch=1, 
                                          epochs=n_epochs)

# --- 5. Training Loop (Unchanged) ---

print("-" * 30)
print(f"Starting Training for {n_epochs} epochs...")
model.train() 

for epoch in range(1, n_epochs + 1):
    for inputs_batch, targets_batch in train_loader:
        # Backward and optimize
        optimizer.zero_grad()
        
        # Forward pass:
        outputs = model(inputs_batch)
        
        # Calculate loss (outputs: [N, 1], targets: [N, 1])
        loss = criterion_primary(outputs, targets_batch)
        
        
        loss.backward()
        optimizer.step()
        scheduler.step()

    if epoch % 1000 == 0 or epoch == 1:
        avg_loss = loss 
        print(f'Epoch [{epoch:05d}/{n_epochs}], Loss: {avg_loss:.6f}')
        model.eval()
        with torch.no_grad():
            val_losses = []
            for val_inputs, val_targets in val_loader:
                val_preds = model(val_inputs)
                val_losses.append(criterion_primary(val_preds,val_targets).item())

        print(f'Validation loss: {np.mean(val_losses):.6f}')
print("-" * 30)
print("Training Complete!")

DNN Architecture built: 961 -> [160, 26, 4, 1]
Model weights initialized with Xavier/Glorot.
Model instantiatied with 158219 parameters.
------------------------------
Starting Training for 5000 epochs...
Epoch [00001/5000], Loss: 4061.400635
Validation loss: 98.617279
Epoch [01000/5000], Loss: 2.279125
Validation loss: 1.501750
Epoch [02000/5000], Loss: 1.264415
Validation loss: 1.392141
Epoch [03000/5000], Loss: 0.039816
Validation loss: 1.225056
Epoch [04000/5000], Loss: 0.003772
Validation loss: 1.200222
Epoch [05000/5000], Loss: 0.002243
Validation loss: 1.196635
------------------------------
Training Complete!


In [98]:
print(val_preds.T, '\n',val_targets.T)

tensor([[ 7.6704,  9.0087,  9.0024,  8.6160, 10.7008,  9.0340,  8.7720,  8.6591]]) 
 tensor([[ 9.5290,  7.0030,  8.6340,  7.4670, 11.5860,  8.8950,  7.3440,  8.9260]])


In [64]:
for name, params in model.named_parameters():
    print(name)

model.fc_stack[(0)].weight[]

fc_stack.0.weight
fc_stack.0.bias
fc_stack.3.weight
fc_stack.3.bias
fc_stack.6.weight
fc_stack.6.bias
fc_stack.8.weight
fc_stack.8.bias


tensor([ 0.0781,  0.0149, -0.0588, -0.0413, -0.0362, -0.0397,  0.0411,  0.0031,
        -0.0608, -0.0208, -0.0488,  0.0084,  0.0646, -0.0818, -0.0814, -0.0193,
        -0.0359, -0.0204,  0.0859, -0.0047, -0.0289, -0.0616, -0.0333,  0.0642,
         0.0704, -0.0371,  0.0234,  0.0768,  0.0118, -0.0679,  0.0206, -0.0010,
        -0.0605, -0.0812, -0.0428, -0.0416, -0.0118, -0.0483,  0.0239, -0.0181,
        -0.0075, -0.0717,  0.0292,  0.0408, -0.0871, -0.0020, -0.0265, -0.0811,
        -0.0300,  0.0800, -0.0514,  0.0386, -0.0690, -0.0437,  0.0354, -0.0230,
        -0.0456,  0.0232, -0.0515, -0.0029, -0.0225,  0.0355,  0.0393, -0.0589,
         0.0884, -0.0167, -0.0537,  0.0057, -0.0103, -0.0388,  0.0207,  0.0920,
        -0.0072,  0.0334, -0.0164,  0.0446, -0.0403, -0.0480, -0.0126,  0.0709,
        -0.0071, -0.0423, -0.0876,  0.0138, -0.0053,  0.0341,  0.0070, -0.0136,
        -0.0116, -0.0288,  0.0078, -0.0319, -0.0338, -0.0095, -0.0179,  0.0280,
         0.0325, -0.0699, -0.0515, -0.00

In [84]:
print(val_preds.T, '\n',val_targets.T)

tensor([[8.6046, 8.6046, 8.6046, 8.6046, 8.6046, 8.6046, 8.6046, 8.6046]]) 
 tensor([[ 9.5290,  7.0030,  8.6340,  7.4670, 11.5860,  8.8950,  7.3440,  8.9260]])
